Import libraries:

In [ ]:
import pandas as pd
import numpy as np
# import tensorflow as tf
import matplotlib.pyplot as plt

Load dataset:

In [ ]:
df = pd.read_csv('dataset.csv')
df.dropna()
df.head()